# Bug / Issue

Boundary conditions that are JIT functions are not correctly handled by the 
saddle point solver. These work for Poisson / Poisson-Vector problems just fine
but something seems to go wrong during the SNES solve for saddle point problems.

Sometimes this seems to be true for sympy functions, sometimes not, but it is 
reproducible if the uw function is a mesh variable of any kind. 

In the example below, this fails for the saddle point system.
```
incompressible_vector_projection.add_dirichlet_bc( (s_soln.fn,0.0), "Bottom" , (1,) )
```
but this works
```
incompressible_vector_projection.add_dirichlet_bc( (0.0,0.0), "Bottom" , (1,) )
```
and apparently, so does a sympy function of x,y that must also be compiled. 

In [1]:
import petsc4py
import underworld3 as uw
import numpy as np

In [2]:
meshbox = uw.meshing.UnstructuredSimplexBox(
                                             minCoords=(0.0,0.0), 
                                             maxCoords=(1.0,1.0), 
                                             cellSize=1.0/32.0, 
                                             regular=True)

In [3]:
# Doesn't matter which element type - this one is also broken !
# meshbox = uw.util_mesh.StructuredQuadBox(
#                                            minCoords=(0.0,0.0), 
#                                            maxCoords=(1.0,1.0), 
#                                            elementRes=(32,32)
# )

In [4]:
import sympy

# Some useful coordinate stuff 

x = meshbox.N.x
y = meshbox.N.y
z = meshbox.N.z

In [5]:
# Project values from a swarm to a mesh variable (with bcs)

swarm  = uw.swarm.Swarm(mesh=meshbox)
s_values  = uw.swarm.SwarmVariable(r"S^{s}", swarm, 1,           proxy_degree=1)
v_values  = uw.swarm.SwarmVariable(r"V^{s}", swarm, meshbox.dim, proxy_degree=1)
iv_values = uw.swarm.SwarmVariable(r"V^{i}", swarm, meshbox.dim, proxy_degree=1)

swarm.populate(fill_param=3)

In [6]:
s_soln  = uw.discretisation.MeshVariable("T",    meshbox,  1,            degree=2 )
v_soln  = uw.discretisation.MeshVariable('U',    meshbox,  meshbox.dim,  degree=2 )
iv_soln = uw.discretisation.MeshVariable(r'U^{i}',   meshbox,  meshbox.dim,  degree=2 )

In [7]:
s_fn = sympy.cos(5.0*sympy.pi * x) * sympy.cos(5.0*sympy.pi * y)
sv_fn = meshbox.vector.gradient(s_fn)

In [8]:
vector_projection = uw.systems.Vector_Projection(meshbox, v_soln)
vector_projection.uw_function = v_values.sym
vector_projection.smoothing = 1.0e-3

# Velocity boundary conditions (compare left / right walls in the soln !)

vector_projection.add_dirichlet_bc( (0.0,), "Left" ,   (0,) )
vector_projection.add_dirichlet_bc( (0.0,), "Right" ,  (0,) )
vector_projection.add_dirichlet_bc( (0.0,), "Top" ,    (1,) )
vector_projection.add_dirichlet_bc( (s_soln.fn, ), "Bottom" , (1,) )

In [148]:
# try to enforce incompressibility

incompressible_vector_projection = uw.systems.Vector_Projection(meshbox, iv_soln)
incompressible_vector_projection.uw_function =  iv_values.sym 
incompressible_vector_projection.smoothing = 1.0e-3
incompressible_vector_projection.penalty = 0.1


# Velocity boundary conditions (compare left / right walls in the soln !)
incompressible_vector_projection.add_dirichlet_bc( (0.0,), "Left" ,   (0,) )
incompressible_vector_projection.add_dirichlet_bc( (0.0,), "Right" ,  (0,) )
incompressible_vector_projection.add_dirichlet_bc( (0.0,), "Top" ,    (1,) )
incompressible_vector_projection.add_dirichlet_bc( (s_soln.fn, ), "Bottom" , (1,) )

In [149]:
with swarm.access(s_values, v_values, iv_values):
    s_values.data[:,0]  = uw.function.evaluate(s_fn, swarm.data, meshbox.N)    
    v_values.data[:,0]  = uw.function.evaluate(sympy.cos(5.0*sympy.pi * x) * sympy.cos(5.0*sympy.pi * y), swarm.data, meshbox.N)    
    v_values.data[:,1]  = uw.function.evaluate(sympy.sin(5.0*sympy.pi * x) * sympy.sin(5.0*sympy.pi * y), swarm.data, meshbox.N)
    iv_values.data[:,0] = uw.function.evaluate(sympy.cos(5.0*sympy.pi * x) * sympy.cos(5.0*sympy.pi * y), swarm.data, meshbox.N)    
    iv_values.data[:,1] = uw.function.evaluate(sympy.sin(5.0*sympy.pi * x) * sympy.sin(5.0*sympy.pi * y), swarm.data, meshbox.N)  

In [150]:
vector_projection.solve()

  0 SNES Function norm 0.0118698 
    Residual norms for VProj1_ solve.
    0 KSP Residual norm 1.392220571848e+01 
    1 KSP Residual norm 4.900683122602e+00 
    2 KSP Residual norm 1.728220770654e+00 
    3 KSP Residual norm 5.757358561194e-01 
    4 KSP Residual norm 1.833726892815e-01 
    5 KSP Residual norm 5.599747892341e-02 
    6 KSP Residual norm 1.864311192547e-02 
    7 KSP Residual norm 6.664052519993e-03 
  1 SNES Function norm 1.13607e-05 
Nonlinear VProj1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1


In [151]:
options = incompressible_vector_projection.petsc_options
options.setValue("pc_gamg_type","agg")
options.setValue("pc_gamg_agg_nsmooths", 3)
options.setValue("pc_gamg_threshold", 0.5)

display(options.getAll())


{'_private_petscspace_degree': '2',
 'ksp_monitor': None,
 'ksp_rtol': '0.001',
 'ksp_type': 'gmres',
 'pc_gamg_agg_nsmooths': '3',
 'pc_gamg_threshold': '0.5',
 'pc_gamg_type': 'agg',
 'pc_type': 'gamg',
 'snes_converged_reason': None,
 'snes_monitor_short': None,
 'snes_rtol': '0.001',
 'snes_type': 'newtonls'}

In [152]:
incompressible_vector_projection.solve(_force_setup=True)

  0 SNES Function norm 0.0855145 
    Residual norms for VProj8_ solve.
    0 KSP Residual norm 1.115778703468e+00 
    1 KSP Residual norm 2.699438660212e-01 
    2 KSP Residual norm 1.783773001972e-01 
    3 KSP Residual norm 1.492752279043e-01 
    4 KSP Residual norm 1.332733836869e-01 
    5 KSP Residual norm 1.268237135425e-01 
    6 KSP Residual norm 1.198304145166e-01 
    7 KSP Residual norm 1.093078849498e-01 
    8 KSP Residual norm 9.802976505269e-02 
    9 KSP Residual norm 8.469861120689e-02 
   10 KSP Residual norm 6.781064273374e-02 
   11 KSP Residual norm 5.562264544260e-02 
   12 KSP Residual norm 4.620714667808e-02 
   13 KSP Residual norm 3.799623848321e-02 
   14 KSP Residual norm 3.230575212587e-02 
   15 KSP Residual norm 2.787760673976e-02 
   16 KSP Residual norm 2.338353018285e-02 
   17 KSP Residual norm 2.029355122979e-02 
   18 KSP Residual norm 1.731700696200e-02 
   19 KSP Residual norm 1.509953581749e-02 
   20 KSP Residual norm 1.301171471401e-02 
   2